# Predictive Maintenance Model 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Loading the dataset
df = pd.read_csv('predictive_maintenance.csv')
print("Data loaded successfully")

Data loaded successfully


In [3]:
# 1. Define Target based on Failure Type
df['Binary_Target'] = df['Failure Type'].apply(lambda x: 0 if x == 'No Failure' else 1)

print("Target Distribution:")
print(df['Binary_Target'].value_counts())

# 2. Define Features
# Drop unneeded columns
drop_cols = ['UDI', 'Product ID', 'Failure Type', 'Target', 'Binary_Target']
X = df.drop(drop_cols, axis=1)
y = df['Binary_Target']

# 3. Encode Categorical 'Type' column
X = pd.get_dummies(X, columns=['Type'], drop_first=True)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nTraining features:", list(X.columns))

Target Distribution:
Binary_Target
0    9652
1     348
Name: count, dtype: int64

Training features: ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Type_L', 'Type_M']


## Model Training (Random Forest)

In [4]:
# Initialize and train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred = rf_model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.983

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      1935
           1       0.90      0.54      0.67        65

    accuracy                           0.98      2000
   macro avg       0.94      0.77      0.83      2000
weighted avg       0.98      0.98      0.98      2000



## Safety Switch Logic
The function below uses the model to decide if a machine should be turned off.

In [6]:
def should_turn_off(sensor_data, model, columns):
    """
    Decides whether to turn off the machine.
    Returns True if the model predicts a failure (Class 1).
    """
    # Convert input dict to DataFrame
    input_df = pd.DataFrame([sensor_data])
    
    # Preprocessing: Encode Type if present
    if 'Type' in input_df.columns:
         input_df = pd.get_dummies(input_df, columns=['Type'], drop_first=True)
    
    # Align columns (add missing dummies as 0)
    for col in columns:
        if col not in input_df.columns:
            input_df[col] = 0
            
    # Reorder columns to match training data
    input_df = input_df[columns]
    
    # Predict
    prediction = model.predict(input_df)[0]
    
    return prediction == 1
